In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

In [3]:
#data = pd.read_csv('./boston_contest.csv')
data = pd.read_csv('./boston_contest_fillna.csv')

In [4]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MDEV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [5]:
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MDEV
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,3.425809,11.911765,11.036513,0.071429,0.551840,6.298645,68.247059,3.842749,9.571429,408.466387,18.467017,355.221218,12.620756,22.773950
std,7.804001,23.668291,6.946327,0.257810,0.114333,0.713019,28.256380,2.127558,8.738369,169.182483,2.179615,93.688016,7.247511,9.367767
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.080050,0.000000,5.085000,0.000000,0.448000,5.884750,44.225000,2.109150,4.000000,278.500000,17.375000,375.782500,6.840000,17.100000
50%,0.250895,0.000000,8.560000,0.000000,0.532000,6.214000,76.600000,3.275900,5.000000,330.000000,19.100000,391.240000,11.300000,21.400000
75%,3.674807,20.000000,18.100000,0.000000,0.624000,6.630250,94.025000,5.222125,24.000000,666.000000,20.200000,395.810000,16.995000,25.525000
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


### Preprocessing

### NA 수 Count

In [6]:
data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MDEV       0
dtype: int64

In [7]:
data['ZN'] = data['ZN'].fillna(0)
data['NOX'] = data['NOX'].fillna(data.NOX.mean())

#### 상관계수

In [8]:
for col in data.columns:
    print(col)
    print(data.corrwith(data[col]))
#print(data.iloc[:, [0, 1, 2]])

CRIM
CRIM       1.000000
ZN        -0.216086
INDUS      0.424929
CHAS      -0.055244
NOX        0.454208
RM        -0.282908
AGE        0.371433
DIS       -0.393464
RAD        0.648849
TAX        0.604748
PTRATIO    0.298684
B         -0.437457
LSTAT      0.481684
MDEV      -0.400791
dtype: float64
ZN
CRIM      -0.216086
ZN         1.000000
INDUS     -0.538711
CHAS      -0.046577
NOX       -0.526979
RM         0.308723
AGE       -0.577162
DIS        0.673172
RAD       -0.322392
TAX       -0.322104
PTRATIO   -0.412658
B          0.183948
LSTAT     -0.416251
MDEV       0.353672
dtype: float64
INDUS
CRIM       0.424929
ZN        -0.538711
INDUS      1.000000
CHAS       0.059189
NOX        0.765611
RM        -0.400302
AGE        0.647328
DIS       -0.707403
RAD        0.598038
TAX        0.722439
PTRATIO    0.414374
B         -0.362375
LSTAT      0.602716
MDEV      -0.482587
dtype: float64
CHAS
CRIM      -0.055244
ZN        -0.046577
INDUS      0.059189
CHAS       1.000000
NOX        0.081

#### RAD, TAX : 0.909147, delete RAD col
#### CHAS : Charles River dummy variable (= 1 if tract bounds river; 0 otherwise), delete CHAS col

In [9]:
data.drop(['RAD'], axis='columns', inplace=True)
data.drop(['CHAS'], axis='columns', inplace=True)
data

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MDEV
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...
471,0.06263,0.0,11.93,0.573,6.593,69.1,2.4786,273,21.0,391.99,9.67,22.4
472,0.04527,0.0,11.93,0.573,6.120,76.7,2.2875,273,21.0,396.90,9.08,20.6
473,0.06076,0.0,11.93,0.573,6.976,91.0,2.1675,273,21.0,396.90,5.64,23.9
474,0.10959,0.0,11.93,0.573,6.794,89.3,2.3889,273,21.0,393.45,6.48,22.0


### Column Def
- ZN : proportion of residential land zoned for lots over 25,000 sq.ft.
- NOX : nitrogen oxides concentration (parts per 10 million).

Column Definition을 보고 na를 뭘로 fill 할 것인지 결정

### fit 하기 전 reshape

In [10]:
total_cols = len(data.columns)
x = data.iloc[:,:-1].values.reshape(-1,total_cols - 1)
y = data.iloc[:,-1]

In [11]:
x

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

### Train set / Test set 분리

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((333, 11), (143, 11), (333,), (143,))

### Scaler

- StandardScaler : 기본 스케일. 평균과 표준편차 사용  
- MinMaxScaler : 최대/최소값이 각각 1, 0이 되도록 스케일링  
- MaxAbsScaler : 최대절대값과 0이 각각 1, 0이 되도록 스케일링  
- RobustScaler : 중앙값(median)과 IQR(interquartile range) 사용. 아웃라이어의 영향을 최소화  


In [13]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [14]:
x_train_scaled

array([[1.61795220e-03, 0.00000000e+00, 2.96920821e-01, ...,
        8.82978723e-01, 9.95814212e-01, 2.87101248e-01],
       [1.14652097e-02, 0.00000000e+00, 2.81524927e-01, ...,
        8.93617021e-01, 9.88980786e-01, 3.30374480e-01],
       [7.18665369e-02, 0.00000000e+00, 6.46627566e-01, ...,
        8.08510638e-01, 9.53300721e-01, 4.67128988e-01],
       ...,
       [1.42128450e-04, 1.75000000e-01, 3.37243402e-02, ...,
        6.38297872e-01, 9.90771093e-01, 1.70041609e-01],
       [5.88534192e-03, 0.00000000e+00, 2.10410557e-01, ...,
        5.10638298e-01, 9.58242977e-01, 5.10402219e-02],
       [1.93022676e-03, 0.00000000e+00, 3.50073314e-01, ...,
        5.53191489e-01, 9.93973473e-01, 2.32454924e-01]])

### Linear Regression Fit

In [15]:
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)
pred = lr.predict(x_test_scaled)

In [16]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, pred)

In [17]:
print(mse)

21.444061418324214


In [18]:
np.sqrt(mse)

4.63077330673012

In [19]:
x_train_fix = []
y_train_fix = []
x_test_fix = []
y_test_fix = []
lr_fix = LinearRegression()
min_mse = 987654321.0
for i in range(10000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
    scaler = MinMaxScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    lr = LinearRegression()
    lr.fit(x_train_scaled, y_train)
    pred = lr.predict(x_test_scaled)
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, pred)
    res = np.sqrt(mse)
    if res < min_mse:
        print(f"minval = {res}")
        min_mse = res
        x_train_fix = x_train
        x_test_fix = x_test
        y_train_fix = y_train
        y_test_fix = y_test
        lr_fix = lr

minval = 5.288806008410983
minval = 4.7077508261728305
minval = 4.483646783151456
minval = 4.366655101952357
minval = 4.309734050579212
minval = 4.294710545003284
minval = 4.064504550660749
minval = 3.926123719838299
minval = 3.837826486582463
minval = 3.801481146951481
minval = 3.79628694475097
minval = 3.600888171880315
minval = 3.532883137461671
minval = 3.4935584206257744
minval = 3.471525272786814
minval = 3.4056284201388785


In [20]:
isANS = False
if isANS:
    ans = pd.read_csv('./answer.csv')
    ans.drop(['RAD'], axis='columns', inplace=True)
    ans.drop(['CHAS'], axis='columns', inplace=True)
    x_ans = ans.iloc[:,:-1].values.reshape(-1,total_cols - 1)
    y_ans = ans.iloc[:,-1]
    scaler_ans = MinMaxScaler()
    scaler_ans.fit(x_ans)
    x_ans_scaled = scaler_ans.transform(x_ans)
    pred_ans = lr_fix.predict(x_ans_scaled)
    mse_ans = mean_squared_error(y_ans, pred_ans)
    print(f"ans result = {np.sqrt(mse_ans)}")

In [21]:
lr_fix.coef_

array([ -3.15159956,   4.02185521,  -2.19942891,  -7.22662702,
        21.02248199,  -0.28384702, -17.97724023,   1.27622273,
        -8.58306929,   4.41866935, -19.34652999])

In [25]:
import array
b = [1, 2, 3]
b = np.array(b)
print(b)

[1 2 3]
